In [10]:
from langchain_community.llms import Ollama

In [11]:
llm = Ollama(model = "llama3.1")

In [12]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_core.prompts import PromptTemplate

In [13]:
db = SQLDatabase.from_uri("postgresql://abouzuhayr:@localhost:5432/postgres")

In [14]:
write_query = create_sql_query_chain(llm=llm, db=db)

In [15]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda

In [17]:
import re
# Define your function to extract and execute the SQL query
def extract_and_execute_sql(inputs):
    # Extract the response text and question
    response = inputs.get('response', '')
    question = inputs.get('question', '')

    # Define the regular expression pattern to match the SQL query
    pattern = re.compile(r'SQLQuery:\s*(.*)')

    # Search for the pattern in the response
    match = pattern.search(response)

    if match:
        # Extract the matched SQL query
        sql_query = match.group(1).strip()

        # Execute the query using your custom logic
        result = db.run(sql_query)

        # Return the necessary information for the next steps in the chain
        return {
            "question": question,
            "query": sql_query,
            "result": result
        }
    else:
        return {
            "question": question,
            "query": None,
            "result": "No SQL query found in the response."
        }

# Wrap your function with RunnableLambda
extract_and_execute = RunnableLambda(extract_and_execute_sql)

In [18]:
from langchain_core.output_parsers import StrOutputParser
# Create a Runnable that wraps write_query and passes along the question
def write_query_with_question(inputs):
    response = write_query.invoke(inputs)
    return {'response': response, 'question': inputs['question']}

write_query_runnable = RunnableLambda(write_query_with_question)

# Create the chain
chain = (
    write_query_runnable
    | extract_and_execute
    | answer_prompt
    | llm
    | StrOutputParser()
)

# Invoke the chain with your question
response = chain.invoke({"question": "How many employees are there in total?"})

# Print the final answer
print(response)

There are 5 employees.
